<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/SML2020/10_logistic_regression_for_text_mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ロジスティック回帰を使ってテキストマイニングを試みる

* WikipediaからUSの男性俳優と女性俳優のページをクローリングして、簡単な前処理をしたデータを使う。

* 分析の目的1: 検証データでできるかぎりチューニングを行い、最後にテストデータでの分類性能を明らかにする。

* 分析の目的2： 男性俳優と女性俳優のページを分類する際に、どのような単語が特に効いているかを調べる。

 * この調査によって、俳優に関する記述におけるジェンダー・ステレオタイプを明らかにできるか？

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### データファイルを読み込む

* データファイルは、あらかじめ自分のGoogle Driveの適当なフォルダに置いておく。

* データファイルの各行には、女性俳優(1)か男性俳優(0)かを表すフラグ、俳優の名前、Wikipediaのページの本文が、この順に格納されている。

* データファイルの各行に対してeval組み込み関数を適用すると、Pythonのリストに変換できるようなフォーマットで、ファイルに記録されている。

In [0]:
y = list()
names = list()
corpus = list()
with open('drive/My Drive/data/us_actors_and_actresses.txt', 'r') as f:
    for line in f:
        flag, name, text = eval(line.strip())
        y.append(int(flag))
        names.append(name)
        corpus.append(text)

In [0]:
y = np.array(y)
names = np.array(names)

In [4]:
N = len(y)
print('We have {:d} documents.'.format(N))

We have 19645 documents.


In [5]:
indices = np.arange(N)
np.random.seed(123)
np.random.shuffle(indices)
print(indices)

[12617 18144 17561 ... 15377 17730 15725]


In [6]:
border = N * 8 // 10
print('We have {:d} test documents.'.format(N - border))
train_indices = indices[:border]
test_indices = indices[border:]
y_train = y[train_indices]
y_test = y[test_indices]
names_train = names[train_indices]
names_test = names[test_indices]

We have 3929 test documents.


In [0]:
corpus_train = list()
for i in train_indices:
  corpus_train.append(corpus[i])
corpus_test = list()
for i in test_indices:
  corpus_test.append(corpus[i])

（ここまでのtraining setとtest setへの分割は、変えないようにしてください。）

### TF-IDFで各文書をベクトル化する

* TF-IDFは単語列をベクトル化する方法のひとつ。

* ベクトルの次元は語彙数となる。各Wikipediaのページがひとつのベクトルへ変換される。

* TfidfVectorizerのパラメータをチューニングしても構わない

 * ここでTF-IDFの計算をするときにテストデータは使っていないので、ズルはしていない。

 * min_dfは、その数より少ない文書にしか出現しない単語を削除する、という意味のパラメータ。希少な単語を削除するために使う。

 * max_dfは、0から1の間の実数で指定すると、その割合より多い文書に出現する単語を削除する、という意味のパラメータ。ありふれた単語を削除するために使う。

In [8]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=100)
X_train = vectorizer.fit_transform(corpus_train)
print('# X_train shape', X_train.shape)

# X_train shape (15716, 6033)


In [0]:
vocab = np.array(vectorizer.get_feature_names())

In [10]:
print(vocab[1000:1010])

['bringing' 'brings' 'britain' 'british' 'broad' 'broadcast'
 'broadcasting' 'broadcasts' 'broadway' 'broderick']


###  交差検証の準備

* ここはk-fold交差検証に書き換えても良い。

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.10, random_state=42)

### ロジスティック回帰の学習

* ロジスティック回帰のハイパーパラメータは、検証データでの分類性能ができるだけよくなるように、変更する。

In [12]:
# これは単純な実行例にすぎません。正則化も含めてチューニングしてください。
# TfidfVectorizerのパラメータも併せてチューニングしていいです。

clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)
print(clf.score(X_valid, y_valid))

0.8829516539440203


### テストデータで最終的な評価をおこなう

In [13]:
X_test = vectorizer.transform(corpus_test)
print('# X_test shape', X_test.shape)
print(clf.score(X_test, y_test))

# X_test shape (3929, 6033)
0.8788495800458132


### 分類に効いている単語を調べる

* 訓練データが最も数が多いので、訓練データの分類に最も効いている単語100語を調べる。

* 下に示すのは、あくまで一つの方法にすぎない。他にどんな方法があるか調べて、その方法を実践する。

 * 下の手法の欠点は、男性俳優の文書に特徴的な単語と、女性俳優の文書に特徴的な単語とを、区別できない点である。

In [14]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=clf, n_features_to_select=200, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

['actresses' 'adam' 'age' 'alice' 'amanda' 'amy' 'andrew' 'angela' 'anime'
 'ann' 'anna' 'anne' 'annie' 'announcer' 'anthony' 'army' 'attack' 'aunt'
 'baby' 'band' 'baseball' 'beauty' 'ben' 'billy' 'birth' 'blonde' 'bobby'
 'boy' 'boys' 'breast' 'brother' 'brothers' 'captain' 'catherine'
 'character' 'charles' 'cheerleader' 'chris' 'christopher' 'cindy'
 'claire' 'contract' 'corps' 'dance' 'dancer' 'daniel' 'daughter' 'dave'
 'david' 'debut' 'detective' 'died' 'directed' 'directing' 'director'
 'divorced' 'dorothy' 'eddie' 'edward' 'elizabeth' 'emily' 'eric' 'eve'
 'fashion' 'father' 'female' 'florence' 'football' 'frances' 'frank'
 'fraternity' 'game' 'gang' 'gay' 'george' 'gina' 'girl' 'girlfriend'
 'girls' 'grace' 'grandmother' 'guitar' 'guy' 'heather' 'helen' 'hero'
 'heroine' 'husband' 'ii' 'irene' 'jack' 'james' 'jane' 'jason' 'jay'
 'jean' 'jeff' 'jennifer' 'jenny' 'jessica' 'jim' 'jimmy' 'joe' 'john'
 'johnny' 'jonathan' 'joseph' 'joy' 'jr' 'julie' 'katie' 'kevin' 'lady'
 'larr